## Data Exploration

### Where Data come from?

### What is the data about?


### When it was collected, and over what time period

### How many rows and columns

### What are the types of the columns

###  Some basic statistics and plots
     a. Plot / Pie with Major distribution
     b. Plot with race distribution
     c. Plot with work_exp
     d. Plot / Pie with work industry
     e. more??


In [ ]:
# Load data

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
%matplotlib inline

url = 'https://raw.githubusercontent.com/toutsos/ml_project/refs/heads/main/MBA.csv'
df=pd.read_csv(url)
# print(df.head())

## Data Cleaning

### How much missing data

### What's the form of missing data?

### Where's the missing data?


### What was done about the missing data?

## Data exploration and visualization

### Various plots that are appropiate for the data.

### For every plot, the question that motivated it and what you learned from it.

## Propose ML approach

### Why specific algorithm/s?

### Is there something about the data that has driven that decision?


### Use at least 2 algorithms per person with solid alalysis of hyperparameters.

## Evaluate Performance

### Why specific metric?

### Are there alternatives?

### Use at least 2 metrics with solid explanation and performance under that metric.

## Present Results.

### Visualize the results.

### Explain plots and results carefully.

### Provide evidence of the results supporting or not supporting the goal.

### Touth on all components in the data science process.

### Did we achieve our goal?

### What we could have done better?

### What else could we do?

### What would be most impactful next step and why?

## Final Report.

### Turn in code and report document (word/pdf).

### Each people should highlight its contribution.